In [19]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)




In [20]:
# conda activate aso_design
import pandas as pd
from scripts.data_genertion.consts import *
from asodesigner.file_utils import read_human_genome_fasta_dict
from asodesigner.consts import *
aso_df = pd.read_csv(DATA_PATH / 'data_asoptimizer_updated.csv')
aso_df.head()

/tmp/ipykernel_1562/3800543916.py:6: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  aso_df = pd.read_csv(DATA_PATH / 'data_asoptimizer_updated.csv')


,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,Inhibition(%),seq_length,Canonical Gene Name,Cell line organism,Transcript,Location_in_sequence,Location_div_by_length,true_length_of_seq,mod_scan,cell_line_uniform
0,0,540733,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCTAAAACAAATGCTA,...,33.0,16,KRAS,human,NaN,41212.0,0.901972,16,0,A431
1,1,540747,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,TATAATGGTGAATATC,...,7.0,16,KRAS,human,NaN,23686.0,0.518395,16,0,A431
2,2,540806,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCATGAAGATTTCTGG,...,62.0,16,KRAS,human,NaN,43363.0,0.949049,16,1,A431
3,3,651479,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GGTGAATATCTTCAAA,...,28.0,16,KRAS,human,NaN,23680.0,0.518264,16,0,A431
4,4,651490,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,CACTTGTACTAGTATG,...,36.0,16,KRAS,human,NaN,41168.0,0.901009,16,0,A431


Reading the A expression file that contain the RNA expression matrix for each cell line

In [21]:
expr = pd.read_csv("OmicsExpressionProteinCodingGenesTPMLogp1.csv")
expr = expr.rename(columns={expr.columns[0]: "DepMap_ID"})
print(expr.head())

    DepMap_ID  TSPAN6 (7105)  TNMD (64102)  DPM1 (8813)  SCYL3 (57147)  \
0  ACH-001113       4.956577      0.000000     7.577648       3.179411   
1  ACH-001289       4.954992      0.617243     7.334747       2.783576   
2  ACH-001339       3.421952      0.000000     7.546069       2.615880   
3  ACH-001979       4.651643      0.000000     5.946408       2.454515   
4  ACH-002438       4.336705      0.000000     6.879387       2.262824   

   FIRRM (55732)  FGR (2268)  CFH (3075)  FUCA2 (2519)  GCLC (2729)  ...  \
0       4.765742    0.037483    1.510547      3.103039     6.697482  ...   
1       3.736280    0.000000    0.149060      3.855960     4.377061  ...   
2       4.476233    0.064571    1.397491      6.833915     3.980336  ...   
3       1.852111    0.000000    7.680796      6.156866     5.042574  ...   
4       3.256491    0.000000    2.253926      6.895436     3.470992  ...   

   H3C3 (8352)  CIMIP3 (114841037)  SMIM42 (117981789)  NPBWR1 (2831)  \
0     0.093516           

loading the Metadata mapping ACH IDs to readable cell line names.

In [22]:
model = pd.read_csv("Model.csv")

Merge the two df

In [23]:
expr_named = expr.merge(
    model[["ModelID", "StrippedCellLineName"]],
    left_on="DepMap_ID", right_on="ModelID", how="left"
)
expr_named = expr_named.drop(columns=["DepMap_ID"])
expr_named = expr_named.rename(columns={"StrippedCellLineName": "cell line"})
cols = ["cell line"] + [c for c in expr_named.columns if c != "cell line"]
expr_named = expr_named[cols]
print(expr_named.head())



       cell line  TSPAN6 (7105)  TNMD (64102)  DPM1 (8813)  SCYL3 (57147)  \
0        LC1SQSF       4.956577      0.000000     7.577648       3.179411   
1       COGAR359       4.954992      0.617243     7.334747       2.783576   
2        COLO794       3.421952      0.000000     7.546069       2.615880   
3          NZM11       4.651643      0.000000     5.946408       2.454515   
4  CCLFPEDS0026T       4.336705      0.000000     6.879387       2.262824   

   FIRRM (55732)  FGR (2268)  CFH (3075)  FUCA2 (2519)  GCLC (2729)  ...  \
0       4.765742    0.037483    1.510547      3.103039     6.697482  ...   
1       3.736280    0.000000    0.149060      3.855960     4.377061  ...   
2       4.476233    0.064571    1.397491      6.833915     3.980336  ...   
3       1.852111    0.000000    7.680796      6.156866     5.042574  ...   
4       3.256491    0.000000    2.253926      6.895436     3.470992  ...   

   CIMIP3 (114841037)  SMIM42 (117981789)  NPBWR1 (2831)  ACTL10 (170487)  \
0  

Reading the catalog of RNA-binding proteins (RBPs) and their motifs

In [24]:
motifs= pd.read_csv("RBS_motifs_Homo_sapiens.csv")

merging all the df

In [25]:
import re
import pandas as pd

# --- 1) Helper: canonicalize gene symbols (strip parentheses, trim, uppercase) ---
def canonical_symbol(s: str) -> str:
    s = str(s).strip()
    # remove trailing " (12345)" or any parentheses block at the end
    s = re.sub(r"\s*\(.*?\)\s*$", "", s)
    # collapse internal whitespace
    s = re.sub(r"\s+", "", s)
    return s.upper()

# --- 2) Melt expression to long and add a canonical gene column ---
expr_long = expr_named.melt(
    id_vars=["cell line"],
    var_name="Gene_raw",
    value_name="Expression"
)
expr_long["Gene_name"] = expr_long["Gene_raw"].apply(canonical_symbol)

# (optional) keep only real genes (drop the 'cell line' row if it leaked)
expr_long = expr_long[expr_long["Gene_name"].ne("CELLLINE")]


# make sure the column exists; if it's differently named, change here
assert "Gene_name" in motifs.columns, "Expected 'Gene_name' column in motifs file."

motifs = motifs.copy()
motifs["Gene_name"] = motifs["Gene_name"].apply(canonical_symbol)



# --- 4) Merge on the canonical symbol ---
merged = expr_long.merge(motifs, on="Gene_name", how="inner")

# sanity check
print("Rows after merge:", len(merged))
print(merged.head())


Rows after merge: 5478052
  cell line      Gene_raw Expression Gene_name          Gene_id Mutated  \
0   LC1SQSF  RBM5 (10181)   5.591937      RBM5  ENSG00000003756      no   
1   LC1SQSF  RBM5 (10181)   5.591937      RBM5  ENSG00000003756      no   
2   LC1SQSF  RBM5 (10181)   5.591937      RBM5  ENSG00000003756      no   
3   LC1SQSF  RBM5 (10181)   5.591937      RBM5  ENSG00000003756      no   
4   LC1SQSF  RBM5 (10181)   5.591937      RBM5  ENSG00000003756      no   

       Organism      Motif  Len  \
0  Homo_sapiens    AAAAAAA    7   
1  Homo_sapiens     AGGUAA    6   
2  Homo_sapiens    CCCCCCC    7   
3  Homo_sapiens  CUCUUCUCU    9   
4  Homo_sapiens    GAAGGAA    7   

                              Experiment_description Database    Pubmed  \
0  Homopolymer binding assay with recombinant pro...        S  11029660   
1                       Chemical shift mapping (NMR)        S  22162216   
2  Homopolymer binding assay with recombinant pro...        S  11029660   
3  EMSA and 

Filter merged table to only ASO-relevant cell lines (only for the cell lines in our ASO data)

In [26]:
import re

def _norm_cell(v: str) -> str:
    # Uppercase + remove spaces/hyphens/underscores/slashes
    s = str(v).upper()
    s = re.sub(r"[\s\-\_/]+", "", s)
    return s

# Unique cell lines from ASO file (normalized)
aso_cell_lines = {_norm_cell(c) for c in aso_df["Cell_line"].dropna()}

# Add A549 manually (normalized)
aso_cell_lines.add(_norm_cell("A549"))

# Normalize the "cell line" column in merged and filter
merged_norm = merged["cell line"].astype(str).map(_norm_cell)
filtered = merged[ merged_norm.isin(aso_cell_lines) ].copy()

print("Original size:", len(merged))
print("Filtered size:", len(filtered))


Original size: 5478052
Filtered size: 29277


save as csv

In [27]:
filtered.to_csv("RBP_motifs_for_each_cell_line_with_mRNA_expression.csv",index=False )